In [1]:
import numpy  as np
import pandas as pd
from scipy.sparse import load_npz

train_x_path='train_x.npz'
train_y_path='train_y_encoded.npz'
test_x_path='test_x.npz'

print 'load in training set...'
train_x=np.array(load_npz(train_x_path).todense())
train_y = np.array(load_npz(train_y_path).todense())

#put random seed here
selected_sample_index=np.random.choice(int(train_x.shape[0]),1000,replace=False)
train_x=train_x[selected_sample_index,]
train_y=train_y[selected_sample_index,]

print 'train_x:',train_x.shape
print 'train_y:',train_y.shape


def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def loss(y,nn_output):
    return 0.5*(sum(np.array(y)-np.array(nn_output))**2)

class NN(object):
    def __init__(self, num_neuron_per_layer,lr=0.01,batch=1,epoch=20):
        self.lr=lr
        self.batch=batch
        self.epoch=epoch
        self.num_layers = len(num_neuron_per_layer)
        self.num_neuron_per_layer = num_neuron_per_layer#dummy neuron for the bias ignored
        self.reset_weights_bias()

    def reset_weights_bias(self):
        self.weights=[]
        self.bias=[]
        for i in range(0,self.num_layers-1):
            weight=np.random.randn(self.num_neuron_per_layer[i], self.num_neuron_per_layer[i+1])
            self.weights.append(weight)
            self.bias.append(np.random.randn(self.num_neuron_per_layer[i+1]))

    #per sample
    def feedforward(self, nn_input):
        layer_output_his=[]
        layer_output_his.append(nn_input)#to facilitate sgd, add the 0-layer input
        layer_input=np.array(nn_input)
        #layer_input.reshape(-1,1)
        for w,b in zip(self.weights,self.bias):
            layer_output = sigmoid(np.dot(w.T, layer_input)+b[0])
            layer_output_his.append(layer_output)
            layer_input=layer_output
        return layer_output_his #layer_output_his[-1] is nn_output
    
    def gradient_descent(self,true_y,layer_output_his):
        delta_weights=[]
        delta_bs=[]
        layer_output=layer_output_his[-1]
        sig=np.dot(layer_output,np.ones(len(layer_output))-layer_output)
        delta_base=np.dot(true_y-layer_output,sig)
        layer_output_his=layer_output_his[:-1]
        for w,layer_output in zip(reversed(self.weights),reversed(layer_output_his)):
            delta_weight=-np.outer(layer_output,delta_base)
            delta_weights.append(delta_weight)
            delta_b=-delta_base
            delta_bs.append(delta_b)
            sig=np.dot(layer_output,np.ones(len(layer_output))-layer_output)
            delta_base=-np.dot(w,np.multiply(sig,delta_base))
        return delta_weights,delta_bs
    
    def batch_gradient_descent(self, input_x,input_y):
        while input_x.shape[0]>self.batch:
            
            current_batch_index=np.random.choice(int(input_x.shape[0]),self.batch,replace=False)
            batch_x=input_x[current_batch_index,]
            batch_y=input_y[current_batch_index,]
            input_x=np.delete(input_x, current_batch_index, axis=0)
            input_y=np.delete(input_y, current_batch_index, axis=0)
            acc_weights=0
            acc_bias=0
            for x,y in zip(batch_x,batch_y):
                layer_output_his=self.feedforward(x)
                delta_weights,delta_bs=self.gradient_descent(y,layer_output_his)
                acc_weights=np.add(acc_weights,delta_weights)
                acc_bias=np.add(acc_bias,delta_bs)
            self.update_model(acc_weights,acc_bias)
        if input_x.shape[0]>0:
            for x,y in zip(input_x,input_y):
                layer_output_his=self.feedforward(x)
                delta_weights,delta_bs=self.gradient_descent(y,layer_output_his)
                acc_weights=np.add(acc_weights,delta_weights)
                acc_bias=np.add(acc_bias,delta_bs)
            self.update_model(acc_weights,acc_bias)
    
    def update_model(self,delta_weights,delta_bs):
        for i in range(0,self.num_layers-1):
            self.weights[i]=np.add(self.weights[i],np.multiply(delta_weights[-(i+1)],self.lr))
            self.bias[i]=np.add(self.bias[i],np.multiply(delta_bs[-(i+1)],self.lr))
    
    def split_dataset(self,input_x,input_y,num_folder=3):
        if len(input_x)!=len(input_y):
            print 'len(input_x) = ',len(input_x),'; len(input_y) = ',len(input_y)
            print 'len(input_x) must be equal to len(input_y).'
            return None
        return (np.linspace(0,len(input_y),num_folder+1,endpoint=True)).astype(int).tolist()[1:]
        
        
    def fit_model(self,input_x,input_y,cv=False):
        if cv:
            split_index_end=self.split_dataset(input_x,input_y)
            mse_his=[]
            start=0
            for end in split_index_end:
                valid_x=input_x[start:end]
                valid_y=input_y[start:end]
                train_x=np.concatenate((input_x[0:start],input_x[end:]))
                train_y=np.concatenate((input_y[0:start],input_y[end:]))
                
                self.reset_weights_bias()
                
                for i in range(0,self.epoch):
                    print 'epoch ',i,' ...'
                    self.batch_gradient_descent(input_x,input_y)
                mse=0
                for x,y in zip(valid_x,valid_y):
                    mse+=loss(y,self.feedforward(x)[-1])
                mse_his.append(float(mse)/len(valid_y))
                start=end
            return np.array(mse_his).mean()
                        
        else:
            train_x=input_x
            train_y=input_y
            
            
            self.reset_weights_bias()
                
            for i in range(0,self.epoch):
                print 'epoch ',i,' ...'
                self.batch_gradient_descent(input_x,input_y)
            
            mse=0
            for x,y in zip(train_x,train_y):
                mse+=loss(y,self.feedforward(x)[-1])
            return float(mse)/len(train_y)
    
    #per sample
    def predict(self,x):
        return np.argmax(self.feedforward(x)[-1])
    
def grid_search_model(input_x,input_y,lr_candidates=[0.01],nn_candidates=[[4096,100,10]]):
    #an exmaple with lr
    #init models
    min_mse=float('inf')
    min_lr=None
    min_cc=None
    c_mse=None
    log=[]
    for c_nn in nn_candidates:
        for c_lr in lr_candidates:
            
            n=NN(num_neuron_per_layer=c_nn,lr=c_lr)
            c_mse=n.fit_model(input_x,input_y,cv=True)
            if min_mse>c_mse:
                min_mse=c_mse
                min_lr=c_lr
                min_nn=c_nn
            print 'best nn=',c_nn,'  c_lr=',c_lr, ' c_mse=',c_mse
            c_log=[]
            c_log.append(c_nn)
            c_log.append(c_lr)
            c_log.append(c_mse)
            log.append(c_log)
    return min_lr,min_nn,log
            

load in training set...
train_x: (1000L, 4096L)
train_y: (1000L, 10L)


In [2]:
lr_candidate =[0.001,0.01,0.1,1]
nn_candidate=[[4096,128,10],[4096,512,10],[4096,1024,10],[4096,2048,10],[4096,128,128,10],[4096,128,512,10],[4096,128,1024,10],[4096,512,128,10],[4096,512,512,10],[4096,512,1024,10]]
#nn_candidate=[[4096,128,10],[4096,512,10],[4096,128,128,10],[4096,128,512,10]]
best_lr,best_nn,log=grid_search_model(train_x,train_y,lr_candidates=lr_candidate,nn_candidates=nn_candidate)


epoch  0  ...
epoch  1  ...
epoch  2  ...
epoch  3  ...
epoch  4  ...
epoch  5  ...
epoch  6  ...
epoch  7  ...
epoch  8  ...
epoch  9  ...
epoch  10  ...
epoch  11  ...
epoch  12  ...
epoch  13  ...
epoch  14  ...
epoch  15  ...
epoch  16  ...
epoch  17  ...
epoch  18  ...
epoch  19  ...
epoch  20  ...
epoch  21  ...
epoch  22  ...
epoch  23  ...
epoch  24  ...
epoch  25  ...
epoch  26  ...
epoch  27  ...
epoch  28  ...
epoch  29  ...
epoch  30  ...
epoch  31  ...
epoch  32  ...
epoch  33  ...
epoch  34  ...
epoch  35  ...
epoch  36  ...
epoch  37  ...
epoch  38  ...
epoch  39  ...
epoch  40  ...
epoch  41  ...
epoch  42  ...
epoch  43  ...
epoch  44  ...


KeyboardInterrupt: 

In [ ]:
print log
df = pd.DataFrame(log)
df.to_csv("tuning_log.csv",header=False,index=False)

In [ ]:
#build final model configured with best hyperparameter on whole training data
n=NN(num_neuron_per_layer=best_nn,lr=best_lr)
f_mse=n.fit_model(train_x,train_y,cv=False)

In [ ]:
#print training accuracy
acc=0
for x,y in zip(train_x,train_y):
    predict_y=n.predict(x)
    if predict_y==np.argmax(y):
        acc+=1
print float(acc)/len(train_x)


In [ ]:
'''
#load in test data
print 'load in dummy test set...'
test_x=train_x[100:]
test_y=train_y[100:]
print 'test_x:',test_x.shape
print 'test_y:',test_y.shape
'''

In [ ]:
'''
#print testing accuracy
acc=0
for x,y in zip(test_x,test_y):
    predict_y=n.predict(x)
    if predict_y==np.argmax(y):
        acc+=1
print float(acc)/len(test_x)
'''

In [ ]:
'''
current_batch_index=np.random.choice(int(train_x.shape[0]),1,replace=False)
print current_batch_index
batch_x=train_x[current_batch_index,]
print batch_x
batch_y=train_y[current_batch_index,]
print batch_y
#input_x=np.delete(train_x, current_batch_index, axis=0)
#input_y=np.delete(train_y, current_batch_index, axis=0)
'''

In [ ]:
#print best_model
print n.lr
print n.num_neuron_per_layer
print n.batch
print n.epoch

In [ ]:
r=pd.read_csv('tuning_log.csv')
r.columns =['nn','lr','loss']

In [ ]:
print r

In [ ]:
r.groupby('nn').groups

In [ ]:
#r.groupby('lr').groups
for lr in lr_candidate:
    print r.groupby('lr').get_group(lr)

In [ ]:
for nn in nn_candidate:
    print r.groupby('nn').get_group(str(nn))

In [ ]:
r.groupby('lr').get_group(0.01).values

In [ ]:
#load in real test data
print 'load in real test set...'
test_x=np.array(load_npz(test_x_path).todense())
result=[]
print 'test_x:',test_x.shape
for index, x in enumerate(test_x):
    predict_y=n.predict(x)
    result.append([index,predict_y])
result_df = pd.DataFrame(result)
result_df.columns =['Id','Label']
result_df.to_csv("test_y_predict.csv",header=True,index=False)